# Cleaning People Dataset

name.basics.tsv.gz file contains the following information for the people in the IMDB database:


- nconst (string) - alphanumeric unique identifier of the name/person
- primaryName (string)– name by which the person is most often credited
- birthYear – in YYYY format
- deathYear – in YYYY format if applicable, else '\N'
- primaryProfession (array of strings)– the top-3 professions of the person
- knownForTitles (array of tconsts) – titles the person is known for

I only need the information on the people that appeared in the movies in my movies dataframe. So I will need to filter the people using movies dataframe.

In [1]:
import gzip
import pandas as pd
import numpy as np
import pickle
import requests

f = open("token_TMDB.txt","r")
token = f.read()
headers = {"accept": "application/json", "Authorization": f"Bearer {token}"}

In [2]:
people = pickle.load(open('name.basics.pkl', 'rb'))

In [3]:
people.shape

(13420451, 6)

## Uploading Cast & Crew Data

People dataset has over 13M rows. I will filter the people data using cast and crew datasets to get rid of the unnecessary data. 

In [4]:
crew = pickle.load(open('crew.pkl', 'rb'))
cast = pickle.load(open('cast.pkl', 'rb'))

In [5]:
people_ids = set(crew['writers'].explode())|set(crew['directors'].explode())|set(cast['nconst'].unique())
len(people_ids)

160556

I have 160k people in my movies dataset so I can filter out the rest from people dataset.

In [6]:
people = people[people['nconst'].isin(people_ids)]

Let's check if there is any person that exists in cast&crew datasets and is missing in people dataset.

In [7]:
filt = list(pd.Series(list(people_ids))[~pd.Series(list(people_ids)).isin(people['nconst'])])
filt

['', 'nm1969711', 'nm2930110']

In [8]:
people_ids = people_ids - set(filt)

There is only 2 people missing in people data. I check for the movies that they appear in our dataset.

In [9]:
filt = [x for x in filt if x != ''] #remove empty string as well
display(crew[crew['writers'].apply(lambda x: any(item in filt for item in x))])
display(crew[crew['directors'].apply(lambda x: any(item in filt for item in x))])
display(cast[cast['nconst'].isin(filt)])

,tconst,directors,writers
386691,tt0399040,"{nm1587451, nm1587401, nm1585207, nm1352003, n...","{nm1587401, nm2707225, nm1352003, nm2930110, n..."
2585450,tt13057626,{nm6213247},{nm1969711}


,tconst,directors,writers


,tconst,ordering,nconst,category


They only appear in 2 movies, so I can easily filter them out.

In [10]:
def filtering(lst, filt):
    filtered = {item for item in lst if item not in filt}
    return filtered if filtered else set()

#try the function on the problematic rows
crew['writers'].apply(lambda x: filtering(x, filt)).loc[[2585450, 386691]]

2585450                                              {}
386691     {nm2707225, nm1352003, nm1587401, nm1552261}
Name: writers, dtype: object

In [11]:
crew['writers'] = crew['writers'].apply(lambda x: filtering(x, filt))

In [12]:
directors_df = crew.explode(column='directors')[['tconst','directors']]
directors_df.rename(columns={'directors': 'nconst'},inplace=True)
directors_df['profession'] = 'director'
directors_df.reset_index(drop=True)

,tconst,nconst,profession
0,tt0029284,nm0437717,director
1,tt0031359,nm0225555,director
2,tt0031976,nm0715346,director
3,tt0032179,nm0212704,director
4,tt0032181,nm0188669,director
...,...,...,...
35753,tt9907782,nm1193346,director
35754,tt9908390,nm1415268,director
35755,tt9911196,nm0631590,director
35756,tt9916270,nm1480867,director


In [13]:
writers_df = crew.explode(column='writers')[['tconst','writers']]
writers_df.rename(columns={'writers': 'nconst'},inplace=True)
writers_df['profession'] = 'writers'
writers_df.reset_index(drop=True)

,tconst,nconst,profession
0,tt0029284,nm0818415,writers
1,tt0029284,nm0437717,writers
2,tt0029284,nm0818416,writers
3,tt0029284,nm0565609,writers
4,tt0029284,nm0855139,writers
...,...,...,...
72752,tt9911196,nm2063122,writers
72753,tt9916270,nm1480867,writers
72754,tt9916270,nm10538402,writers
72755,tt9916362,nm3471432,writers


In [14]:
cast_df = cast[['tconst','nconst','category']]
cast_df.rename(columns={'category': 'profession'},inplace=True)
cast_df.reset_index(drop=True)

/var/folders/_8/f5zjdbxn0vs8y_m3qv7n_8800000gn/T/ipykernel_10655/1085031761.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cast_df.rename(columns={'category': 'profession'},inplace=True)


,tconst,nconst,profession
0,tt0029284,nm0002050,actress
1,tt0029284,nm0000026,actor
2,tt0029284,nm0000068,actor
3,tt0029284,nm0665850,actress
4,tt0029284,nm0794700,actress
...,...,...,...
308167,tt9916362,nm10678596,actress
308168,tt9916362,nm10678595,actress
308169,tt9916362,nm10678597,actress
308170,tt9916362,nm3084436,actor


In [15]:
relations = pd.concat([cast_df,directors_df,writers_df])
relations.dropna(inplace=True)
relations = relations[relations['nconst'].isin(people['nconst'])]
relations.reset_index(drop=True)

,tconst,nconst,profession
0,tt0029284,nm0002050,actress
1,tt0029284,nm0000026,actor
2,tt0029284,nm0000068,actor
3,tt0029284,nm0665850,actress
4,tt0029284,nm0794700,actress
...,...,...,...
415930,tt9911196,nm2063122,writers
415931,tt9916270,nm1480867,writers
415932,tt9916270,nm10538402,writers
415933,tt9916362,nm3471432,writers


In [17]:
relations.to_pickle('cast_crew.pkl')

In [16]:
del directors_df, writers_df, cast_df

In [29]:
people

,nconst,primaryName,birthYear,deathYear,primaryProfession
0,nm0000001,Fred Astaire,1899.0,1987.0,"actor,miscellaneous,producer"
1,nm0000002,Lauren Bacall,1924.0,2014.0,"actress,soundtrack,archive_footage"
2,nm0000003,Brigitte Bardot,1934.0,NaN,"actress,music_department,producer"
3,nm0000004,John Belushi,1949.0,1982.0,"actor,writer,music_department"
4,nm0000005,Ingmar Bergman,1918.0,2007.0,"writer,director,actor"
...,...,...,...,...,...
13419545,nm9992630,Karin Cherches,NaN,NaN,actress
13419614,nm9992750,Loghan Bazan,NaN,NaN,actress
13420288,nm9993535,Henry Lawfull,2006.0,NaN,actor
13420358,nm9993616,Ryan Mac Lennan,NaN,NaN,actor


In [18]:
people.reset_index(drop=True)

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899.0,1987.0,"actor,miscellaneous,producer","tt0072308,tt0050419,tt0053137,tt0027125"
1,nm0000002,Lauren Bacall,1924.0,2014.0,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,1934.0,NaN,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949.0,1982.0,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918.0,2007.0,"writer,director,actor","tt0050986,tt0083922,tt0069467,tt0050976"
...,...,...,...,...,...,...
160548,nm9992630,Karin Cherches,NaN,NaN,actress,tt5039088
160549,nm9992750,Loghan Bazan,NaN,NaN,actress,tt8736336
160550,nm9993535,Henry Lawfull,2006.0,NaN,actor,"tt10187208,tt5900600"
160551,nm9993616,Ryan Mac Lennan,NaN,NaN,actor,tt4844148


# Creating knownFor dataset

Each person might have multiple movies that they are known for, so I will create a dataset to link the people to the movies in our database for which they are known. 

In [19]:
movies = pickle.load(open('movies.pkl', 'rb'))

In [20]:
known_for = people.apply(lambda row: [(row['nconst'], title) for title in row['knownForTitles'].split(',')] 
                         if isinstance(row['knownForTitles'], str) else [], axis=1)
known_for = pd.DataFrame([item for sublist in known_for for item in sublist], columns=['nconst', 'knownForTitle'])
known_for = known_for[known_for['knownForTitle'].isin(movies['tconst'])]
known_for.reset_index(drop=True)

,nconst,knownForTitle
0,nm0000001,tt0072308
1,nm0000001,tt0050419
2,nm0000001,tt0053137
3,nm0000002,tt0037382
4,nm0000002,tt0075213
...,...,...
237287,nm9992630,tt5039088
237288,nm9992750,tt8736336
237289,nm9993535,tt10187208
237290,nm9993616,tt4844148


In [21]:
known_for.to_pickle('known_for.pkl')

In [22]:
people.drop(columns='knownForTitles', inplace=True)

In [23]:
people.to_pickle('people.pkl')

## Exporting datasets to MySQL

I export known for, people and relations directly to SQL from this notebook using sqlalchemy. I created a schema called recommender already and I will add my datasets as tables in this schema.

In [24]:
from sqlalchemy import create_engine
from getpass import getpass

In [25]:
password = getpass()

········


In [26]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost:3306/recommender'
engine = create_engine(connection_string)

In [27]:
with engine.connect() as conn:
    relations.to_sql(name='relations', con=engine, if_exists='replace', index=False)
    people.to_sql(name='people', con=engine, if_exists='replace', index=False)
    known_for.to_sql(name='knownFor', con=engine, if_exists='replace', index=False)